In [3]:
import networkx as nx
import torch_geometric
import torch

from itertools import combinations

# make it autoreload when do changes in py files
%load_ext autoreload
%autoreload 2
from topobenchmarkx.data.liftings.liftings import SimplicialNeighborhoodLifting

In [4]:
# mutang dataset from torch geometric
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root="/tmp/MUTAG", name="MUTAG")

In [5]:
lift = SimplicialNeighborhoodLifting()
data_lst = [lift(data) for data in dataset]

AttributeError: 'TUDataset' object has no attribute 'split'

In [5]:
# cora dataset from torch geometric
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root="/tmp/Cora", name="Cora")

In [6]:
lift = SimplicialNeighborhoodLifting()

data = lift(dataset.data)

RuntimeError: [enforce fail at alloc_cpu.cpp:75] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 5239850621184 bytes. Error code 12 (Cannot allocate memory)

In [ ]:
data

In [17]:
len(data)

2

Processing...
Done!


In [21]:
# mutang dataset from torch geometric
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root="/tmp/MUTAG", name="MUTAG")

Processing...
Done!


In [33]:
type(dataset)

torch_geometric.datasets.tu_dataset.TUDataset